The very first step is to download and install the FETPAPI wheel (which depends on FESAPI) : pip install fetpapi

Once FETPAPI wheel installed, you need to first import FESAPI (required dependency) and then fetpapi it in order to be able to call its API.

In [98]:
import fesapi
import fetpapi

Let's now connect to an ETP server by defining some few initialization parameters such as the URL and the authorization

In [99]:
import uuid

etp_server_url = "wss://..."
authorization = "Bearer ..."
initialization_params = fetpapi.InitializationParameters(str(uuid.uuid4()), etp_server_url)

Some ETP servers such as the OSDU RDDMS one require some xtra header fields. Let's add them

In [100]:
additionalHeaderField = fetpapi.MapStringString()
additionalHeaderField["data-partition-id"] = "osdu"
initialization_params.setAdditionalHandshakeHeaderFields(additionalHeaderField)

We can now create the ETP session.

In [101]:
client_session = fetpapi.createClientSession(initialization_params, authorization)

And we can finally run the session in parallel not to block the main thread

In [102]:
def start_etp_server(client_session):
    client_session.run()

from threading import Thread
t = Thread(target=start_etp_server, args=(client_session,), daemon=True)
t.start()

Let's wait for the connection to be effective

In [103]:
from time import sleep, perf_counter
start_time = perf_counter()
while client_session.isEtpSessionClosed() and perf_counter() - start_time < 5:
    sleep(0.25)	
if client_session.isEtpSessionClosed():
    print("The ETP session could not be established in 5 seconds.")
print("Now connected to ETP Server")

Now connected to ETP Server


Let's list the available datspaces

In [104]:
all_dataspaces = client_session.getDataspaces()
if all_dataspaces.empty() :
    print("There is no dataspace on this ETP server.")
for dataspace in all_dataspaces:
    print(dataspace.uri)

eml:///dataspace('demo/Volve')
eml:///dataspace('pwls/3.0')
eml:///dataspace('pwls/copy')


Let's list the content of the first dataspace

In [105]:
dataspace = all_dataspaces.front()
etp_context = fetpapi.ContextInfo()
etp_context.uri = dataspace.uri
etp_context.depth = 1
all_resources = client_session.getResources(etp_context, fetpapi.ContextScopeKind__self);
if all_resources.empty() :
    print("There is no resource in dataspace " + dataspace.uri)
for resource in all_resources:
    print(resource.uri)

Working on dataspace eml:///dataspace('demo/Volve')
eml:///dataspace('demo/Volve')/resqml20.obj_CategoricalProperty(08a1886d-805b-41c7-9be4-d4dc89bc5cf7)
eml:///dataspace('demo/Volve')/resqml20.obj_CategoricalProperty(503ffa63-00d9-4c46-90e3-805308d61376)
eml:///dataspace('demo/Volve')/resqml20.obj_ContinuousProperty(017d1c1d-09aa-42d8-9b3d-ee89fd7b12a9)
eml:///dataspace('demo/Volve')/resqml20.obj_ContinuousProperty(0319e539-c9a5-43bc-a293-338e8f678ef5)
eml:///dataspace('demo/Volve')/resqml20.obj_ContinuousProperty(03960ef6-15b9-4797-b904-cf2af4e7f103)
eml:///dataspace('demo/Volve')/resqml20.obj_ContinuousProperty(039fe124-8843-40a1-a04c-ff65d56f1279)
eml:///dataspace('demo/Volve')/resqml20.obj_ContinuousProperty(0a2e7f32-f085-4350-89a7-95cb6c10b7d8)
eml:///dataspace('demo/Volve')/resqml20.obj_ContinuousProperty(0b533bdf-76f8-4d23-953b-9766fac540cd)
eml:///dataspace('demo/Volve')/resqml20.obj_ContinuousProperty(0db437bf-952c-4cd1-8450-337b4c91f35d)
eml:///dataspace('demo/Volve')/resqml

We can now close the session

In [106]:
client_session.close();